In [96]:
import pandas as pd
import numpy as np
import re
from pandasql import sqldf

In [97]:
df = pd.read_csv('E:/NewDrive/DataScienceProjects/DataSets/Flipkart_dataset/Apple_new.csv')

In [98]:
df.duplicated().sum()

11

In [99]:
df_A= df.drop_duplicates()

In [100]:
pd.options.mode.chained_assignment = None  # default='warn'

In [101]:
df_A['Name']=df_A['Company Name'].str.extract('(APPLE)')

In [102]:
df_A['test'] = df_A['Company Name'].str.extract('(\([^)]*\,)')
df_A['test1']=df_A['test'].astype(str)
df_A['Color'] = df_A['test1'].apply(lambda x: x.replace('(','').replace(',','').replace('nan','Not Mentioned'))

In [103]:
df_A['test2'] = df_A['Company Name'].str.split('(').str[0]
df_A['Model']= df_A['test2'].str.split().str[1:].str.join(' ')

In [104]:
def match(x):
    if re.search("Ratings",x):
        return x
    else:
        return "0"

In [105]:
df_A['test3']=df_A['Ratings'].apply(match)
df_A['test5']=df_A['test3'].apply(lambda x: x[x.find("SAMSUNG") : x.find("Ratings")])
df_A['Ratings(*)']=df_A['test3'].str.extract('(4\.\d|3\.\d)')

In [106]:
df_A['test4']=df_A['test3'].apply(lambda x: x[x.find("Ratings") : x.find("Reviews")])
df_A['Reviews']=df_A['test4'].apply(lambda x: x.replace('Ratings','').replace('&','').replace(',',''))
df_A['Reviews']=df_A['Reviews'].str.strip()

In [107]:
def remove_whitespace(s):
    return ' '.join(s.split())

In [108]:
df_A['MobilesSold']=df_A['Reviews'].apply(remove_whitespace)
df_A['MobilesSold']=df_A['MobilesSold'].replace('','0')
df_A['MobilesSold']=df_A['MobilesSold'].astype(int)

In [109]:
df_A['test6']=df_A['Mobile Price'].apply(lambda x: x.replace('%','').replace('offFree','').replace('delivery','').replace('off','').replace('Free delivery','').replace('Free',''))

In [110]:
df_A['OfferInPerentage'] =df_A['offer'].str.extract('(\d+)')
df_A['OfferInPerentage']=df_A['OfferInPerentage'].fillna(0)
df_A['OfferInPerentage']=df_A['OfferInPerentage'].astype(int)

In [111]:
df_A['test7']=df_A['test6'].str.extract('(\₹[^)]*\₹)')

In [112]:
df_A['test7'] = np.where(df_A['test7'].isnull(),df_A['test6'],df_A['test7'])

In [113]:
df_A['DiscountedPrice']=df_A['test7'].apply(lambda x:str(x).replace('₹','').replace(',',''))
#Replacing NAN Values from Price(with discount) column
df_A['DiscountedPrice']=df_A['DiscountedPrice'].replace('nan','0')
df_A['DiscountedPrice']=df_A['DiscountedPrice'].astype(int)

In [114]:
df_A['MaxRetailPrice'] = (df_A.DiscountedPrice + ((df_A.OfferInPerentage/100) * df_A.DiscountedPrice))
df_A['MaxRetailPrice']=df_A['MaxRetailPrice'].round(0)

In [115]:
df_A['Sales(₹ CR)']=(df_A.DiscountedPrice * df_A.MobilesSold )/10000000
df_A['Sales(₹ CR)']=df_A['Sales(₹ CR)'].round(3)

In [116]:
#df_A = df_A.assign(RAM(GB)=None)
df_A['RAM(GB)'] = 0
df_A['ROM(GB)']= df_A['Specifications'].str.split('ROM').str[0]
df_A['ROM(GB)']=df_A['ROM(GB)'].str.replace('GB','')
df_A['ROM(GB)']=df_A['ROM(GB)'].str.strip()

In [117]:
def case_1(value):
  if value[-2:] == 'MB':
    return float(value[:-2])/1000
  elif value[-2:] == 'TB':
    return float(value[:-2])*1000
  else:
    return value

In [118]:
df_A['ROM(GB)']=df_A['ROM(GB)'].map(lambda x: case_1(x))

In [119]:
df_A['Display']= df_A['Specifications'].str.extract('\((.*?)\)')
df_A['Display']=df_A['Display'].str.replace('inch','')

In [120]:
df_A['Screen']=df_A['Specifications'].str.extract('(XDR|HD)')
df_A['Screen']=df_A['Screen'].fillna("Not Mentioned")

In [121]:
df_A['test9']= df_A['Specifications'].apply(lambda x: x[x.find("Display"):x.find("Front")])

In [122]:
df_A['test10'] = df_A['test9'].apply(lambda x:str(x).replace('Display','').replace('+',''))

In [123]:
df_A[['Rear Camera','Front Camera']]= df_A['test10'].str.split('|',expand = True)
df_A['Rear Camera']= df_A['Rear Camera'].str.replace('Rear Camera','')

In [124]:
df_A['No of Rear Camera']= df_A['Rear Camera'].str.count("MP")
df_A['No of Front Camera']= df_A['Front Camera'].str.count("MP")

In [125]:
df_A['Battery']= df_A['Specifications'].str.extract('(\d{4} mAh)')
df_A['Battery']= df_A['Battery'].fillna("Not Mentioned")

In [126]:
df_A['Processor']= df_A['Specifications'].str.extract('(Bionic Chip|6 Core|A8 Chip with 64-bit|A9 64-bit|A8 Chip with 64-bit|A7 64-bit)')
df_A['Processor']=df_A['Processor'].str.replace('with','')

In [127]:
# drop all test columns and unwanted columns
df_A.drop(columns=['test','test1','test2','test3','test4','test5','test6','test7','test9','test10','Company Name','Ratings','Specifications','Mobile Price','offer'],inplace=True,axis=1)

In [128]:
df_A.to_csv('E:/NewDrive/DataScienceProjects/DataSets/Flipkart_dataset/AppleCleaned.csv',index=False)